# Scalable Signature-Based Distribution Regression via Reference Sets

## 2nd-order Sig-MMD Approximator: Dataset

### Andrew Alden, Blanka Horvath, Carmine Ventre

### Table of Contents

- [Initial Setup](#initial-setup)
- [2nd-order Sig-MMD](#mmd)
- [Zero Distances](#zero-dist)
- [Test Dataset](#test)

## Initial Setup <a class="anchor" id="initial-setup"></a>

### Change working directory to root folder

In [1]:
import os
os.getcwd()
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [2]:
import warnings
warnings.filterwarnings("ignore", "You are using `torch.load` with `weights_only=False`*.")

### Import libraries 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from torch import nn
import torch.cuda
from tqdm.auto import tqdm
import torch
import iisignature

from src.MMD.mmd import RBFKernel, SigKernel

from src.StochasticProcesses.rBergomi import r_bergomi_sample_paths_functional_central_limit
from src.StochasticProcesses.Heston import heston_sample_paths_inv
from src.StochasticProcesses.BlackScholes import GBM_sample_paths

from src.utils import save_dataset, load_dataset, save_path_params, load_path_params


from src.RegressionModel.MMDDataset import MMDDataset, save_checkpoint


In [4]:
import sys

sys.modules['numpy._core'] = np.core
sys.modules['numpy._core.multiarray'] = np.core.multiarray

### Set device

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')
torch.backends.cudnn.benchmark = True

Device: cuda


## 2nd-order Sig-MMD <a class="anchor" id="mmd"></a>

### Setup

In [6]:
static_kernel_21 = RBFKernel(sigma=0.5)
static_kernel_22 = RBFKernel(sigma=0.5)

dyadic_order = 0

signature_kernel = SigKernel([static_kernel_21, static_kernel_22], dyadic_order)

### Compute Distances

In [ ]:
training_dataset = MMDDataset(signature_kernel, 5000, 5000, 5000, 5000, 5000, 5000, device)
training_dataset.compute_distance_path_pairs()
training_dataset.compute_expected_sigs()

save_dataset(training_dataset.rbergomi_rbergomi_sample_paths, 
             training_dataset.rbergomi_rbergomi_distances,
             'MMDApproxData/Datasets/rbergomi_rbergomi_sample_paths', 
             'MMDApproxData/Datasets/rbergomi_rbergomi_distances')

save_dataset(training_dataset.rbergomi_heston_sample_paths, 
             training_dataset.rbergomi_heston_distances, 
             'MMDApproxData/Datasets/rbergomi_heston_sample_paths', 
             'MMDApproxData/Datasets/rbergomi_heston_distances')

save_dataset(training_dataset.rbergomi_gbm_sample_paths, 
             training_dataset.rbergomi_gbm_distances, 
             'MMDApproxData/Datasets/rbergomi_gbm_sample_paths',
             'MMDApproxData/Datasets/rbergomi_gbm_distances')

save_dataset(training_dataset.heston_heston_sample_paths, 
             training_dataset.heston_heston_distances, 
             'MMDApproxData/Datasets/heston_heston_sample_paths',
             'MMDApproxData/Datasets/heston_heston_distances')

save_dataset(training_dataset.heston_gbm_sample_paths, 
             training_dataset.heston_gbm_distances, 
             'MMDApproxData/Datasets/heston_gbm_sample_paths', 
             'MMDApproxData/Datasets/heston_gbm_distances')

save_dataset(training_dataset.gbm_gbm_sample_paths, 
             training_dataset.gbm_gbm_distances, 
             'MMDApproxData/Datasets/gbm_gbm_sample_paths', 
             'MMDApproxData/Datasets/gbm_gbm_distances')

save_checkpoint(training_dataset.xi_0_list, 
                training_dataset.nu_list,
                training_dataset.rho_rbergomi_list,
                training_dataset.H_list, 
                training_dataset.v0_rbergomi_list,
                training_dataset.vol_of_vol_list, 
                training_dataset.speed_list, 
                training_dataset.mean_volatility_list, 
                training_dataset.v0_heston_list, 
                training_dataset.rho_heston_list, 
                training_dataset.sigma_list, 
                training_dataset.r_list, 
                training_dataset.rbergomi_rbergomi_sample_paths,
                training_dataset.rbergomi_rbergomi_distances, 
                training_dataset.rbergomi_heston_sample_paths, 
                training_dataset.rbergomi_heston_distances, 
                training_dataset.rbergomi_gbm_sample_paths, 
                training_dataset.rbergomi_gbm_distances, 
                training_dataset.heston_heston_sample_paths,
                training_dataset.heston_heston_distances,
                training_dataset.heston_gbm_sample_paths, 
                training_dataset.heston_gbm_distances, 
                training_dataset.gbm_gbm_sample_paths, 
                training_dataset.gbm_gbm_distances)


expected_sigs = training_dataset.expected_sigs

distances = training_dataset.rbergomi_rbergomi_distances + training_dataset.rbergomi_heston_distances + training_dataset.rbergomi_gbm_distances + training_dataset.heston_heston_distances + training_dataset.heston_gbm_distances + training_dataset.gbm_gbm_distances

## Zero Distances <a class="anchor" id="zero-dist"></a>

In [ ]:
def compute_expected_sigs(sample_paths, sigma=1, sig_level=4):
    """
    Compute the expected signatures.
    :param sample_paths: Array/List of sample paths of shape [Num Samples, 2, Num Time Steps, Num Sim, 2].
    :param sigma: Sigma hyperparameter.
    :param sig_level: Signature truncation level.
    :return: List of expected signatures.
    """
    
    expceted_sigs = np.mean(iisignature.sig(np.exp(-np.divide(np.power(np.transpose(np.asarray(sample_paths), (0, 1, 3, 2, 4)), 2), sigma)), sig_level), axis=2)
    
    return expceted_sigs.reshape(*expceted_sigs.shape[:-2], -1)

In [15]:
num_same_features = 10000

same_expected_sigs = []
same_distances = [0 for _ in range(num_same_features)]

xi_0_range = (0.01, 0.2)
nu_range = (0.5, 4.0)
rho_range = (-1, 1)
H_range = (0.025, 0.5)
v0_range = (0.2, 0.8)

vol_of_vol_range = (0.2, 0.8)
speed_range = (0.2, 0.8)
mean_volatility_range = (0.2, 0.8)
r_range = (0.01, 0.2)

sigma_range = (0.2, 0.8)

sig_level = 4

S0 = 1

T = 1.0

num_sim = 400
num_time_steps = 14

zero_sample_paths = []

for _ in tqdm(range(num_same_features)):
    
            
    rand = np.random.randint(1, 4, 1)

    if rand[0] == 1:

        p_xi_0 = np.random.uniform(xi_0_range[0], xi_0_range[1])
        p_nu = np.random.uniform(nu_range[0], nu_range[1])
        p_rho = np.random.uniform(rho_range[0], rho_range[1])
        p_H = np.random.uniform(H_range[0], H_range[1])
        p_v0 = np.random.uniform(v0_range[0], v0_range[1])

        p1 = r_bergomi_sample_paths_functional_central_limit(S0, p_v0, p_H, [p_xi_0 for _ in range(num_time_steps+1)], np.sqrt(2*p_H), p_nu, p_rho, T, num_time_steps, num_sim)[-1]
        p2 = r_bergomi_sample_paths_functional_central_limit(S0, p_v0, p_H, [p_xi_0 for _ in range(num_time_steps+1)], np.sqrt(2*p_H), p_nu, p_rho, T, num_time_steps, num_sim)[-1]

    elif rand[0] == 2:

        p_v0 = np.random.uniform(v0_range[0], v0_range[1])
        p_r = np.random.uniform(r_range[0], r_range[1])
        p_rho = np.random.uniform(rho_range[0], rho_range[1])
        p_mean_vol = np.random.uniform(mean_volatility_range[0], mean_volatility_range[1])
        p_speed = np.random.uniform(speed_range[0], speed_range[1])
        p_vol_of_vol = np.random.uniform(vol_of_vol_range[0], vol_of_vol_range[1])

        p1 = heston_sample_paths_inv(S0, p_v0, p_r, p_rho, p_mean_vol, p_speed, p_vol_of_vol, T, num_sim, num_time_steps)[0]
        p2 = heston_sample_paths_inv(S0, p_v0, p_r, p_rho, p_mean_vol, p_speed, p_vol_of_vol, T, num_sim, num_time_steps)[0]

    else:

        p_sigma = np.random.uniform(sigma_range[0], sigma_range[1])
        p_r = np.random.uniform(r_range[0], r_range[1])

        p1 = GBM_sample_paths(S0, p_r, p_sigma, T, num_sim, num_time_steps)
        p2 = GBM_sample_paths(S0, p_r, p_sigma, T, num_sim, num_time_steps)

    zero_sample_paths.append([p1, p2])

100%|██████████| 10000/10000 [00:15<00:00, 636.00it/s]


In [16]:
same_expected_sig = compute_expected_sigs(zero_sample_paths)

In [18]:
expected_sigs = np.concatenate((expected_sigs, same_expected_sig), axis=0)
distances += same_distances

In [21]:
save_dataset(expected_sigs, distances, 'MMDApproxData/expected_sigs_exp', 'MMDApproxData/distances')

## Test Dataset <a class="anchor" id="test"></a>

In [6]:
static_kernel_21 = RBFKernel(sigma=0.5)
static_kernel_22 = RBFKernel(sigma=0.5)

dyadic_order = 0

signature_kernel = SigKernel([static_kernel_21, static_kernel_22], dyadic_order)

In [ ]:
testing_dataset = MMDDataset(signature_kernel, 2, 2, 2, 2, 2, 2, device)
testing_dataset.compute_distance_path_pairs()
testing_dataset.compute_expected_sigs()

expected_sigs = testing_dataset.expected_sigs

distances = testing_dataset.rbergomi_rbergomi_distances + testing_dataset.rbergomi_heston_distances + testing_dataset.rbergomi_gbm_distances + testing_dataset.heston_heston_distances + testing_dataset.heston_gbm_distances + testing_dataset.gbm_gbm_distances

In [ ]:
save_dataset(expected_sigs, distances, 'MMDApproxData/expected_sigs_exp_test', 'MMDApproxData/distances_test')